# Code for training and testing of Mask3D on .las forest plots

Since the data is not yet currently available, I've kept the outputs of the code to demonstrate that it does actually work!


# Installs: (this takes about 1 hour)

This complicated install setup is necessary to get things running on Colab

In [ ]:
%%capture

!git clone "https://github.com/ruarimh/Mask3D"
%cd /content/Mask3D

!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu116.html

!pip install ninja==1.10.2.3
!pip install pytorch-lightning==1.7.2
!pip install fire==0.4.0
!pip install imageio==2.21.1
!pip install tqdm==4.64.0
!pip install wandb==0.13.2
!pip install python-dotenv==0.20.0
!pip install pyviz3d==0.2.28
!pip install scipy==1.9.0
!pip install plyfile==0.7.4
!pip install scikit-learn==1.1.2
!pip install trimesh==3.14.0
!pip install loguru==0.6.0
!pip install albumentations==1.2.1
!pip install volumentations==0.1.8
!pip install laspy

!pip install antlr4-python3-runtime==4.8
!pip install black==21.4b2
!pip install omegaconf==2.0.6 hydra-core==1.0.5 --no-deps
!pip install 'git+https://github.com/facebookresearch/detectron2.git@710e7795d0eeadf9def0e7ef957eea13532e34cf' --no-deps

!pip install git+https://github.com/NVIDIA/MinkowskiEngine.git -v

!pip install -r requirements.txt

%cd third_party/pointnet2
!python setup.py install

%cd /content/Mask3D

"""
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu116.html
"""

!pip install omegaconf==2.0.6 hydra-core==1.0.5 --no-deps
!cp "/content/Mask3D/third_party/pytorch/types.py" "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/types.py"

# Get .las data for all the plots

In [ ]:
# import your Google Drive, assuming your data is stored there

import os
from google.colab import drive
drive.mount('/content/drive')
drive_folder = '/content/drive/MyDrive'

Mounted at /content/drive


In [ ]:
!mkdir /content/Mask3D/data/
!mkdir /content/Mask3D/data/las/
!mkdir /content/Mask3D/data/las/train
!mkdir /content/Mask3D/data/las/validation
!mkdir /content/Mask3D/data/las/test

# copy the FORinstance_dataset onto Colab
!cp -R /content/drive/MyDrive/MRes/Data/FORinstance_dataset/ /content/Mask3D/data/

# Training Mask3D

In [ ]:
# split the data into the train/validation/test subsets
import random
import pandas as pd
import shutil

data_location = "/content/Mask3D/data/FORinstance_dataset/"
train_location = "/content/Mask3D/data/las/train/"
validation_location = "/content/Mask3D/data/las/validation/"
test_location = "/content/Mask3D/data/las/test/"

validation_plots = ["plot59_annotated.las", "plot26_annotated.las",
                    "plot21_annotated.las", "plot8_annotated.las",
                    "plot20_annotated.las", "plot57_annotated.las",
                    "plot12_annotated.las"]

data_split_metadata = pd.read_csv(data_location + "data_split_metadata.csv")
data_split_metadata = data_split_metadata[data_split_metadata["folder"] == "NIBIO2"]

for path in data_split_metadata["path"]:
    if data_split_metadata.loc[data_split_metadata["path"] == path]["split"].item() == "train":
        shutil.copyfile(data_location + path, train_location + path.replace("NIBIO2/", ""))
    else:
        shutil.copyfile(data_location + path, test_location + path.replace("NIBIO2/", ""))

train_files = os.listdir(train_location)

for train_file in train_files:
    if train_file in validation_plots:
        shutil.copyfile(train_location + train_file, validation_location + train_file)
        os.remove(train_location + train_file)

In [ ]:
# make sure you run the code from within the Mask3D directory
%cd /content/Mask3D/

In [ ]:
# run the preprocessing script on the .las files

!python -m datasets.preprocessing.las_preprocessing preprocess \
--data_dir="/content/Mask3D/data/las" \
--save_dir="/content/Mask3D/data/processed/las" \
--sample_proportion=1 \
--use_rgb=True \
--subplot_size=500.0

2023-06-29 18:20:57.613 | INFO     | datasets.preprocessing.base_preprocessing:preprocess:51 - Tasks for train: 28
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:   11.1s finished
2023-06-29 18:

In [ ]:
# log in to weights and biases to track experiment -- not required
!wandb login

Below is the main training script. The output logs are there to show that it actually works

In [ ]:
%%shell
export OMP_NUM_THREADS=3

CURR_DBSCAN=14.0
CURR_TOPK=10
CURR_QUERY=160
CURR_SIZE=250

EXPERIMENT_NAME="all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_norgb"
SAVE_DIRECTORY="/content/drive/MyDrive/MRes/saved/"

# TRAIN
python main_instance_segmentation.py \
general.experiment_name=${EXPERIMENT_NAME} \
general.project_name="las" \
data/datasets=las \
general.num_targets=2 \
data.num_labels=2 \
data.voxel_size=8.0 \
data.num_workers=0 \
data.cache_data=true \
data.cropping_v1=false \
data.cropping=false \
data.batch_size=8 \
data.test_batch_size=1 \
general.reps_per_epoch=5 \
model.num_queries=${CURR_QUERY} \
general.use_dbscan=false \
general.on_crops=true \
model.config.backbone._target_=models.Res16UNet18B \
data.crop_length=${CURR_SIZE} \
data.subplot_size=500.0 \
general.eval_inner_core=-1 \
general.save_visualizations=false \
general.save_dir="${SAVE_DIRECTORY}${EXPERIMENT_NAME}" \
trainer.check_val_every_n_epoch=10 \
trainer.max_epochs=500

/content/Mask3D/trainer/trainer.py:825: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(False, 'class not known!')
2023-06-27 12:32:41.705425: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 2773444861
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 2773444861
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'ram212', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: Currently logged in as: ram212. Use `wandb login --relo

# Mask3D inference and saving visualizations

In [ ]:
# testing visualization data split
# this puts the testing files into the validation folder -- this is necessary
# because the Mask3D test script does not work as intended

import random
import pandas as pd
import shutil

data_location = "/content/Mask3D/data/FORinstance_dataset/"
train_location = "/content/Mask3D/data/las/train/"
validation_location = "/content/Mask3D/data/las/validation/"
test_location = "/content/Mask3D/data/las/test/"

data_split_metadata = pd.read_csv(data_location + "data_split_metadata.csv")
data_split_metadata = data_split_metadata[data_split_metadata["folder"] == "NIBIO2"]

train_count = 0
for path in data_split_metadata["path"]:
    if data_split_metadata.loc[data_split_metadata["path"] == path]["split"].item() == "train":
        shutil.copyfile(data_location + path, train_location + path.replace("NIBIO2/", ""))
    else:
        shutil.copyfile(data_location + path, validation_location + path.replace("NIBIO2/", ""))

train_files = os.listdir(train_location)

validation_plots = ["plot59_annotated.las", "plot26_annotated.las",
                    "plot21_annotated.las", "plot8_annotated.las",
                    "plot20_annotated.las", "plot57_annotated.las",
                    "plot12_annotated.las"]

for train_file in train_files:
    if train_file in validation_plots:
        os.remove(train_location + train_file)

In [ ]:
# make sure you run the code from within the Mask3D directory
%cd /content/Mask3D/

In [ ]:
# run the preprocessing script on the .las files

!python -m datasets.preprocessing.las_preprocessing preprocess \
--data_dir="/content/Mask3D/data/las" \
--save_dir="/content/Mask3D/data/processed/las" \
--sample_proportion=1 \
--use_rgb=True \
--subplot_size=500.0

In [ ]:
# log in to weights and biases to track experiment -- not required
!wandb login

This is the script for producing the testing metrics, and also for getting the files for visualizing the predicted masks.

In [ ]:
%%shell
export OMP_NUM_THREADS=3

CURR_DBSCAN=14.0
CURR_TOPK=10
CURR_QUERY=160
CURR_SIZE=250

EXPERIMENT_NAME="all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb"
SAVE_DIRECTORY="/content/saved/"

# VIZ TEST
python main_instance_segmentation.py \
general.experiment_name=${EXPERIMENT_NAME} \
general.project_name="las" \
data/datasets=las \
general.num_targets=2 \
data.num_labels=2 \
data.voxel_size=6.0 \
data.num_workers=0 \
data.cache_data=true \
data.cropping_v1=false \
data.cropping=false \
data.batch_size=1 \
data.test_batch_size=1 \
general.reps_per_epoch=5 \
model.num_queries=${CURR_QUERY} \
general.use_dbscan=false \
general.on_crops=true \
model.config.backbone._target_=models.Res16UNet18B \
data.crop_length=${CURR_SIZE} \
data.subplot_size=500.0 \
general.eval_inner_core=-1 \
general.save_visualizations=true \
general.save_dir="${SAVE_DIRECTORY}${EXPERIMENT_NAME}" \
general.checkpoint="/content/drive/MyDrive/MRes/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_norgb/last-epoch.ckpt" \
trainer.check_val_every_n_epoch=1 \
optimizer.lr=0.0000000000000001 \
trainer.max_epochs=1

2023-06-27 16:03:24.665493: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 288011894
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 288011894
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'ram212', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: Currently logged in as: ram212. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.13.2

In [ ]:
!zip -r /content/test_files_processed.zip /content/Mask3D/data/processed/las/validation

  adding: content/Mask3D/data/processed/las/validation/ (stored 0%)
  adding: content/Mask3D/data/processed/las/validation/plot52_annotated.npy (deflated 84%)
  adding: content/Mask3D/data/processed/las/validation/plot53_annotated_0.npy (deflated 80%)
  adding: content/Mask3D/data/processed/las/validation/plot58_annotated_0.npy (deflated 79%)
  adding: content/Mask3D/data/processed/las/validation/plot27_annotated.npy (deflated 84%)
  adding: content/Mask3D/data/processed/las/validation/plot10_annotated.npy (deflated 84%)
  adding: content/Mask3D/data/processed/las/validation/plot6_annotated.npy (deflated 85%)
  adding: content/Mask3D/data/processed/las/validation/plot15_annotated.npy (deflated 84%)
  adding: content/Mask3D/data/processed/las/validation/plot27_annotated_0.npy (deflated 79%)
  adding: content/Mask3D/data/processed/las/validation/plot60_annotated.npy (deflated 84%)
  adding: content/Mask3D/data/processed/las/validation/plot34_annotated.npy (deflated 85%)
  adding: content

In [ ]:
!zip -r /content/test_viz_norgb.zip /content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations

  adding: content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations/ (stored 0%)
  adding: content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations/plot3_annotated.las_0.txt_sorted_masks.npy (deflated 100%)
  adding: content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations/plot1_annotated.las_0.txt_original_normals.npy (deflated 100%)
  adding: content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations/plot1_annotated.las_0.txt_sorted_masks.npy (deflated 99%)
  adding: content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations/plot58_annotated.las_0.txt_original_colors.npy (deflated 61%)
  adding: content/saved/all_plots_8.0_voxel_newotherclass_8batch_500crop_8batch_500epoch_viz_test_norgb/visualizations/plot52_annotated.las_0.txt_targ

In [ ]:
!cp /content/test_viz_norgb.zip /content/drive/MyDrive